# Ручное тестирование модели

## Импорты

In [1]:
import numpy as np
import pandas as pd
import json
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

print(f'Библиотеки ипортированы:')
print('tensorflow', tf.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)

Библиотеки ипортированы:
tensorflow 2.19.0
pandas 2.3.0
numpy 2.1.3


## Конфигурация модели

In [2]:
# Пути к файлам
MODEL_PATH = "../model/model.keras"
TOKENIZER_PATH = "../model/tokenizer/tokenizer.json"
CSV_PATH = "../data/processed/dataset.csv"
MAX_LEN = 25

## Проверка конфигурации на наличие файлов

In [3]:
# Проверка наличия каждого файла
missing_files = []

if not os.path.isfile(MODEL_PATH):
    missing_files.append(MODEL_PATH)
if not os.path.isfile(TOKENIZER_PATH):
    missing_files.append(TOKENIZER_PATH)
if not os.path.isfile(CSV_PATH):
    missing_files.append(CSV_PATH)

# Вывод результата
if missing_files:
    print("Ошибка! Не найдены следующие файлы:")
    for file in missing_files:
        print(f"   - {file}")
else:
    print(f"Модель импортирована из пути: {MODEL_PATH}")
    print(f"Токенизатор импортирован из пути: {TOKENIZER_PATH}")
    print(f"Датасет импортирован из пути: {CSV_PATH}")
    print(f"Конфигурация гиперпараметра MAX_LEN установлена: {MAX_LEN}")

Модель импортирована из пути: ../model/model.keras
Токенизатор импортирован из пути: ../model/tokenizer/tokenizer.json
Датасет импортирован из пути: ../data/processed/dataset.csv
Конфигурация гиперпараметра MAX_LEN установлена: 25


In [10]:
# Загрузка модели
# Проверка существования файла перед загрузкой
if os.path.isfile(MODEL_PATH):
    print(f"Файл модели найден: {MODEL_PATH}")
    model = load_model(MODEL_PATH)
    print("Модель успешно загружена.")
else:
    raise FileNotFoundError(f"айл модели не найден по пути: {MODEL_PATH}")

Файл модели найден: ../model/model.keras
Модель успешно загружена.


In [11]:
# Загрузка токенизатора
# Проверка существования файла перед загрузкой токенизатора
if os.path.isfile(TOKENIZER_PATH):
    print(f"Файл токенизатора найден: {TOKENIZER_PATH}")
    with open(TOKENIZER_PATH, "r", encoding="utf-8") as f:
        tokenizer = tokenizer_from_json(f.read())
    print("Токенизатор успешно загружен.")
else:
    raise FileNotFoundError(f"Файл токенизатора не найден по пути: {TOKENIZER_PATH}")

Файл токенизатора найден: ../model/tokenizer/tokenizer.json
Токенизатор успешно загружен.


In [12]:
# Загрузка словаря исправлений
# Проверка существования CSV-файла перед загрузкой
if os.path.isfile(CSV_PATH):
    print(f"CSV-файл найден: {CSV_PATH}")
    df_check = pd.read_csv(CSV_PATH)
    
    # Проверка наличия нужных столбцов
    if "noisy" in df_check.columns and "correct" in df_check.columns:
        corrections = dict(zip(df_check["noisy"], df_check["correct"]))
        print("Словарь исправлений успешно загружен.")
    else:
        raise ValueError("В файле отсутствуют необходимые столбцы: 'noisy' и 'correct'")
else:
    raise FileNotFoundError(f"CSV-файл не найден по пути: {CSV_PATH}")

CSV-файл найден: ../data/processed/dataset.csv
Словарь исправлений успешно загружен.


## Создание блока кода ручного тестирование

In [18]:
# Ручной ввод
print("=== Проверка текста ===")

test_input = "spainish" 

# Преобразуем текст
seq = tokenizer.texts_to_sequences([test_input])
padded = pad_sequences(seq, maxlen=MAX_LEN, padding="post")

# Получаем предсказание
pred = model.predict(padded)[0][0]
print(f"📊 Вероятность ошибки: {pred:.3f}")

# Вывод результата
if pred >= 0.5:
    print("⚠️ Обнаружена ошибка!")
    if test_input in corrections:
        print(f"Вы ввели: {test_input}")
        print(f"✅ Исправление: {corrections[test_input]}")
    else:
        print("❌ Исправление не найдено в словаре.")
else:
    print("✅ Ошибок не обнаружено.")


=== Проверка текста ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
📊 Вероятность ошибки: 0.631
⚠️ Обнаружена ошибка!
Вы ввели: spainish
✅ Исправление: Spanish
